https://github.com/ledmaster/TutorialEnsemble/blob/master/HomePrices.ipynb

In [58]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from itertools import product
from sklearn.linear_model import Ridge

import warnings
warnings.filterwarnings('ignore')

In [31]:
train = pd.read_csv('datasets/train.csv', index_col='Id')

In [32]:
x, y = train.drop('SalePrice', axis=1), train.SalePrice.copy()

In [43]:
# funçoes para calculo do erro

def rmsle(estimator, x, y):
    p = estimator.predict(x)
    return np.sqrt(mean_squared_error(np.log1p(y), np.log1p(p)))

def rmsle_log_y(estimator, x, y):
    p = estimator.predict(x)
    return np.sqrt(mean_squared_error(y,p))

def rmsle_sqrt_y(estimator, x, y):
    p = estimator.predict(x)
    y = np.power(y,2)
    p = np.power(p,2)
    return np.sqrt(mean_squared_error(np.log1p(y), np.log1p(p)))

kf = KFold(n_splits=5, shuffle=True, random_state=1)

In [34]:
x1 = x.select_dtypes(include=[np.number]).fillna(-1)
print('Dims', x1.shape)

model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, x1, y, cv=kf, scoring=rmsle)
print('RMSLE: ', np.mean(error))

Dims (1460, 36)
RMSLE:  0.14582352617986846


### Feature set 2: Ordinal Encoding Categóricas  
Agora vamos criar um outro conjunto de features, desta vez adicionando as variáveis categóricas. Existem várias maneiras de codificar este tipo de variável para os modelos, uma delas é usando um formato ordinal. Isso simplesmente significa substituir cada valor original por números sequenciais. Em alguns modelos isso pode ser problemático, pois eles tentarão capturar alguma relação de ordem em valores que podem não ter. No nosso caso, com modelos baseados em árvores de decisão, este problema é quase inexistente.

In [35]:
x2 = x.copy()

for col in x2.columns:
    if x2[col].dtype == object:
        enc = LabelEncoder()
        x2[col] = enc.fit_transform(x[col].fillna('Missing'))

print('Dims', x2.shape)
x2.fillna(-1, inplace=True)
model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, x2, y, cv=kf, scoring=rmsle).mean()
print('RMSLE: ',error)

Dims (1460, 79)
RMSLE:  0.14383736485915208


### Transformações do Target
Uma maneira interessante de criar diversidade, e às vezes até obter uma melhor performance, num caso de regressão, é transformar a variável que estamos tentando prever. Neste caso testaremos duas transformações: logaritmo e raiz quadrada.

### Log
É possível ver que tentar prever o logaritmo do preço nos dá um resultado melhor. Isto acontece não só pelo fato do modelo capturar padrões diferentes, mas também porque usamos uma métrica baseada na diferença de logaritmos.

In [49]:
model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, x1, np.log1p(y), cv=kf, scoring=rmsle_log_y).mean()
print('RF, x1, log-target RMSLE: ', error)

model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, x2, np.log1p(y), cv=kf, scoring=rmsle_log_y).mean()
print('RF, x2, log-target RMSLE: ', error)

RF, x1, log-target RMSLE:  0.14516215148522235
RF, x2, log-target RMSLE:  0.14213588563678017


### Raiz Quadrada
Esta transformação também nos dá um resultado melhor do que usar a variável em seu estado original. Uma das sugestões da razão pela qual vemos este efeito é que estas transformações fazem com que a variável y tenha uma distribuição mais próxima da normal, o que facilita o trabalho do modelo.

In [52]:
model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, x1, np.sqrt(y), cv=kf, scoring=rmsle_sqrt_y).mean()
print('RF, x1, log-target RMSLE: ', error)

model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, x2, np.sqrt(y), cv=kf, scoring=rmsle_sqrt_y).mean()
print('RF, x2, log-target RMSLE: ', error)

RF, x1, log-target RMSLE:  0.14565293448427202
RF, x2, log-target RMSLE:  0.14300460013198157


### Gerando modelos com modelos/algoritmos diferentes
Outra maneira de gerar diversidade para o ensemble é gerar modelos diferentes. Neste caso vou usar meu modelo preferido, o GBM. Este também é baseado em árvores de decisão, mas basicamente treina cada árvore sequencialmente focando nos erros cometidos pelas anteriores.

Nas células abaixo é possível ver a performance deste modelo nos feature sets e transformações que usamos com a Random Forest. Vemos que ele traz uma melhora significativa, capturando melhor os padrões da relação entre as variáveis e o preço de venda dos imóveis.

In [53]:
model = GradientBoostingRegressor(random_state=0)
error = cross_val_score(model, x1, np.log1p(y), cv=kf, scoring=rmsle_log_y).mean()
print('GBM, x1, log-target RMSLE: ', error)

model = GradientBoostingRegressor(random_state=0)
error = cross_val_score(model, x2, np.log1p(y), cv=kf, scoring=rmsle_log_y).mean()
print('GBM, x1, log-target RMSLE: ', error)

GBM, x1, log-target RMSLE:  0.1334924549135666
GBM, x1, log-target RMSLE:  0.12980689048155078


In [54]:
model = GradientBoostingRegressor(random_state=0)
error = cross_val_score(model, x1, np.sqrt(y), cv=kf, scoring=rmsle_sqrt_y).mean()
print('GBM, X1, sqrt-target RMSLE:', error)

model = GradientBoostingRegressor(random_state=0)
error = cross_val_score(model, x2, np.sqrt(y), cv=kf, scoring=rmsle_sqrt_y).mean()
print('GBM, X2, sqrt-target RMSLE:', error)

GBM, X1, sqrt-target RMSLE: 0.13425897281342522
GBM, X2, sqrt-target RMSLE: 0.1309192356821107


### Stacking

O Stacking é uma maneira de fazer o ensemble na qual usamos modelos para fazer previsões, e depois usamos estas previsões como features em novos modelos, no que pode ser chamado de "segundo nível". Você pode fazer este processo várias vezes, mas a cada nível o retorno em performance com relação à computação necessária é menor.

In [59]:
kf_out = KFold(n_splits=5, shuffle=True, random_state=1)
kf_in = KFold(n_splits=5, shuffle=True, random_state=2)

cv_mean = []
for fold, (tr, ts) in enumerate(kf_out.split(x,y)):
    x1_train, x1_test = x1.iloc[tr], x1.iloc[ts]
    x2_train, x2_test = x2.iloc[tr], x2.iloc[ts]
    y_train, y_test = y.iloc[tr], y.iloc[ts]
    
    modelos = [GradientBoostingRegressor(random_state=0), RandomForestRegressor(random_state=0)]
    targets = [np.log1p, np.sqrt]
    feature_sets = [(x1_train, x1_test), (x2_train, x2_test)]
    
    predictions_cv = []
    predictions_test = []
    for model, target, feature_set in product(modelos, targets, feature_sets):
        predictions_cv.append(cross_val_predict(model, feature_set[0], target(y_train), 
                                                cv=kf_in).reshape(-1,1))
        model.fit(feature_set[0], target(y_train))
        ptest = model.predict(feature_set[1])
        predictions_test.append(ptest.reshape(-1,1))
        
    predictions_cv = np.concatenate(predictions_cv, axis=1)
    predictions_test = np.concatenate(predictions_test, axis=1)
    
    stacker = Ridge()
    stacker.fit(predictions_cv, np.log1p(y_train))
    error = rmsle_log_y(stacker, predictions_test, np.log1p(y_test))
    cv_mean.append(error)
    print('RMSLE Fold %d - RSLE %.4f' % (fold, error))

print("RMSL CV5 %.4f" % np.mean(cv_mean))    

RMSLE Fold 0 - RSLE 0.1250
RMSLE Fold 1 - RSLE 0.1446
RMSLE Fold 2 - RSLE 0.1253
RMSLE Fold 3 - RSLE 0.1387
RMSLE Fold 4 - RSLE 0.1086
RMSL CV5 0.1284
